In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
import os
import time

import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

In [8]:
# Create a new visualizer
vis = meshcat.Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [10]:
vis.jupyter_cell()

In [11]:
finger_base = vis["finger_base"]
finger_joint = finger_base["finger_joint"]
finger = finger_joint["finger"]
eef = finger["eef"]
ellipsoid = vis["ellipsoid"]

finger_base.set_object(g.Box([.05, .05, .05]))
finger.set_object(g.Box([.5, 0.025, 0.025]))
eef.set_object(g.Sphere(0.025), g.MeshLambertMaterial(color=0xff22dd, reflectivity=0.8))
ellipsoid.set_object(g.Ellipsoid([.25, 0.05, .15]), g.MeshLambertMaterial(color=0x5a23cd, reflectivity=0.8))

finger_base.set_transform(tf.translation_matrix([0, 0, .5]))
finger.set_transform(tf.translation_matrix([-.25, 0, 0]))
finger_joint.set_transform(tf.rotation_matrix(-np.pi/2, [1, 0, 0]))
eef.set_transform(tf.translation_matrix([-.25, 0, 0]))



In [13]:
for x in np.linspace(-np.pi, np.pi, 200):
    finger_base.set_transform(tf.translation_matrix([.5 * x / np.pi, 0, .5]))
    finger_joint.set_transform(tf.rotation_matrix(-x / 4 - np.pi / 2, [0, 1, 0]))
    ellipsoid.set_transform(tf.rotation_matrix(-x / 4 - np.pi / 2, [0, 1, 0]))
    time.sleep(0.005)